In [1]:
%pip install dependency-injector pydantic pyathena pyyaml pyiceberg sqlalchemy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 11.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 40.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 41.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from src.keepice_lakehouse import IcebergManagerFactory

factory = IcebergManagerFactory()

spark_manager = factory.get_manager('spark_iceberg')

print("Spark Iceberg Manager created successfully!")

spark_manager.create_database(database_name='test')


Spark Iceberg Manager created successfully!


24/07/28 09:23:57 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [8]:
schema_dict = {
    "VendorID": "bigint",
    "tpep_pickup_datetime": "timestamp",
    "tpep_dropoff_datetime": "timestamp",
    "passenger_count": "double",
    "trip_distance": "double",
    "RatecodeID": "double",
    "store_and_fwd_flag": "string",
    "PULocationID": "bigint",
    "DOLocationID": "bigint",
    "payment_type": "bigint",
    "fare_amount": "double",
    "extra": "double",
    "mta_tax": "double",
    "tip_amount": "double",
    "tolls_amount": "double",
    "improvement_surcharge": "double",
    "total_amount": "double",
    "congestion_surcharge": "double",
    "airport_fee": "double"
}


In [12]:
spark_manager.create_table(database_name='test', table_name='taxi_test_table', columns=schema_dict, s3_folder_location="s3://warehouse/test/taxi-test-table", partition_column = "days(tpep_pickup_datetime)")

In [14]:
print(spark_manager.list_databases().show())
print(spark_manager.list_tables(database_name='nyc').show())
df = spark_manager.get_table_ddl(database_name='nyc', table_name='taxis')
ddl = df.select('createtab_stmt').rdd.flatMap(lambda x: x).collect()[0]
print(ddl)
print(spark_manager.list_tables(database_name='test').show())
df = spark_manager.get_table_ddl(database_name='test', table_name='taxi_test_table')
ddl = df.select('createtab_stmt').rdd.flatMap(lambda x: x).collect()[0]
print(ddl)

athena_manager = factory.get_manager('athena')

print("Athena Manager created successfully!")

pyiceberg_manager = factory.get_manager('pyiceberg')

print("PyIceberg Manager created successfully!")

+---------+
|namespace|
+---------+
|      nyc|
|     test|
+---------+

None
+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|      nyc|    taxis|      false|
|      nyc|     test|      false|
+---------+---------+-----------+

None
CREATE TABLE demo.nyc.taxis (
  VendorID BIGINT,
  tpep_pickup_datetime TIMESTAMP,
  tpep_dropoff_datetime TIMESTAMP,
  passenger_count DOUBLE,
  trip_distance DOUBLE,
  RatecodeID DOUBLE,
  store_and_fwd_flag STRING,
  PULocationID BIGINT,
  DOLocationID BIGINT,
  payment_type BIGINT,
  fare_amount DOUBLE,
  extra DOUBLE,
  mta_tax DOUBLE,
  tip_amount DOUBLE,
  tolls_amount DOUBLE,
  improvement_surcharge DOUBLE,
  total_amount DOUBLE,
  congestion_surcharge DOUBLE,
  airport_fee DOUBLE)
USING iceberg
PARTITIONED BY (days(tpep_pickup_datetime))
LOCATION 's3://warehouse/nyc/taxis'
TBLPROPERTIES (
  'current-snapshot-id' = '7576149768058347624',
  'format' = 'iceberg/parquet',
  'format-version' = '2',


AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `demo`.`test`.`taxis_test_table` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 18;
'ShowCreateTable false, [createtab_stmt#760]
+- 'UnresolvedTableOrView [demo, test, taxis_test_table], SHOW CREATE TABLE, false


In [3]:
spark_manager.create_table(database_name='nyc', table_name='test', columns={"col1":"string"}, s3_folder_location="s3://warehouse/")

In [4]:
print(spark_manager.list_tables(database_name='nyc').show())


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|      nyc|    taxis|      false|
|      nyc|     test|      false|
+---------+---------+-----------+

None


In [14]:
%%sql
TRUNCATE TABLE nyc.taxis;


++
||
++
++

In [5]:
for filename in [
    "yellow_tripdata_2022-04.parquet",
    "yellow_tripdata_2022-03.parquet",
    "yellow_tripdata_2022-02.parquet",
    "yellow_tripdata_2022-01.parquet",
    "yellow_tripdata_2021-12.parquet",
]:
    df = spark.read.parquet(f"/home/iceberg/data/{filename}")
    df.createOrReplaceTempView("temporal_table")
    spark_manager.insert_incremental_table_data(source_table="temporal_table", database_name="nyc", table_name="taxis")

In [6]:
%%sql
SELECT count(*) FROM nyc.taxis;

24/07/28 08:54:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


count(1)
15885533


In [7]:
for filename in [
    "yellow_tripdata_2021-04.parquet",
    "yellow_tripdata_2021-07.parquet"
]:
    df = spark.read.parquet(f"/home/iceberg/data/{filename}")
    df.createOrReplaceTempView("temporal_table")
    spark_manager.insert_bulk_table_data(source_table="temporal_table", database_name="nyc", table_name="taxis")



In [4]:
%%sql
SELECT count(*) FROM nyc.taxis;

count(1)
2821746


In [3]:
for filename in [
    "yellow_tripdata_2021-04.parquet",
    "yellow_tripdata_2021-07.parquet"
]:
    df = spark.read.parquet(f"/home/iceberg/data/{filename}")
    df.createOrReplaceTempView("temporal_table")
    spark_manager.insert_bulk_table_data(source_table="temporal_table", database_name="test", table_name="taxi_test_table")



Py4JJavaError: An error occurred while calling o36.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: NotFoundException: Location does not exist: s3://warehouse/test/taxi-test-table/metadata/00000-ae71a977-9da0-4c64-af66-03a40fef837e.metadata.json
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:217)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:118)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:102)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:201)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:313)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:252)
	at org.apache.iceberg.rest.HTTPClient.get(HTTPClient.java:348)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:96)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadInternal(RESTSessionCatalog.java:331)
	at org.apache.iceberg.rest